In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np

In [2]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import re
from urllib.request import urlopen

# ロジスティック回帰で予想

In [3]:
race_results = pd.read_csv('race_results.csv',encoding="shift jis",index_col=0)

In [4]:
race_results

,race_id,着順,枠番,馬番,馬名,斤量,騎手,タイム,上り,単勝,人気,性,年齢,体重,体重変化
0,200801010101,1,4,6,ディアジーナ,54.0,四位洋文,1:31.0,35.6,4.3,2,牝,2,468,0
1,200801010101,2,8,14,アイディンパワー,54.0,藤田伸二,1:31.4,35.9,5.2,4,牡,2,460,0
2,200801010101,3,1,1,ウインリーズン,54.0,安藤勝己,1:31.5,36.4,3.1,1,牡,2,442,2
3,200801010101,4,7,11,ヴァンエボン,54.0,小林徹弥,1:31.6,35.5,23.0,8,牡,2,482,-4
4,200801010101,5,2,2,ラヴェリータ,54.0,長谷川浩,1:31.6,35.7,21.9,6,牝,2,484,-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,202110040812,7,2,2,ジオルティ,51.0,角田大和,2:42.0,35.3,49.6,10,牡,3,502,8
7,202110040812,8,7,9,ロックグラス,57.0,幸英明,2:42.2,35.8,43.7,8,牡,4,464,0
8,202110040812,9,7,8,サマーカナロア,53.0,富田暁,2:42.2,35.4,46.5,9,牡,3,502,-10
9,202110040812,10,8,10,レッドレイル,57.0,浜中俊,2:42.4,36.3,14.0,6,セ,5,436,2


In [5]:
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression

- アンダーサンプリング
    - 今回のようにrank=4のデータが多いと、モデルが4を予測すれば高精度になるのであまりいいモデルとは言えないものができてしまう
    - データの量をそろえて行うことで精度を上げる方法

In [6]:
#4着以下を全て4にする
clip_rank = lambda x: x if x<4 else 4
race_results['rank'] = race_results['着順'].map(clip_rank)
#不要なカラムを削除
race_results.drop(['着順','馬名','タイム','上り'], axis=1, inplace=True)
#カテゴリ変数をダミー変数化
race_results_d = pd.get_dummies(race_results)

In [7]:
#訓練データとテストデータに分ける
X = race_results_2021_d.drop(["rank"], axis=1)
y = race_results_2021_d["rank"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=0
)

In [8]:
#アンダーサンプリング
rank_1 = y_train.value_counts()[1]
rank_2 = y_train.value_counts()[2]
rank_3 = y_train.value_counts()[3]

rus = RandomUnderSampler(
    sampling_strategy={1: rank_1, 2: rank_2, 3: rank_3, 4: rank_1},
    random_state=71
)

X_train_rus, y_train_rus = rus.fit_resample(X_train.values, y_train.values)

In [9]:
#訓練
model = LogisticRegression()
model.fit(X_train_rus, y_train_rus)

#スコアを表示
print(model.score(X_train, y_train), model.score(X_test, y_test))

0.7814521710348148 0.78143649512041


In [10]:
#予測結果を確認
y_pred = model.predict(X_test)
pred_df = pd.DataFrame({"pred": y_pred, "actual": y_test})
pred_df[pred_df["pred"] == 1]["actual"].value_counts()

Series([], Name: actual, dtype: int64)

In [11]:
#回帰係数の確認
coefs = pd.Series(model.coef_[0], index=X.columns).sort_values()
coefs[["枠番", "馬番", "斤量", "単勝", "人気", "年齢", "体重", "体重変化"]]

枠番      2.499574e-24
馬番     -4.807340e-25
斤量      6.018829e-24
単勝     -9.858802e-22
人気     -9.695998e-23
年齢     -4.755252e-24
体重      1.484160e-22
体重変化    8.101444e-24
dtype: float64

# ランダムフォレストで予想

In [12]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier

In [13]:
race_infos_2021 = pd.read_csv('2021race_infos.csv',encoding="shift jis",index_col=0)
race_results_2021 = pd.read_csv('2021race_results.csv',encoding="shift jis",index_col=0)

In [14]:
#レース結果とレース情報をjoinする
race_results_infos= race_results_2021.merge(race_infos_2021, on='race_id', how='inner')

In [15]:
#データフレームの日付の型を変更する
race_results_infos["date"] = pd.to_datetime(race_results_infos["date"], format="%Y年%m月%d日")

In [16]:
#時系列に沿ってデータを分割
def split_data(df, test_size):
    sorted_id_list = race_results_infos.sort_values("date").index.unique()
    train_id_list = sorted_id_list[: round(len(sorted_id_list) * (1 - test_size))]
    test_id_list = sorted_id_list[round(len(sorted_id_list) * (1 - test_size)) :]
    train = df.loc[train_id_list]
    test = df.loc[test_id_list]
    return train, test

In [17]:
race_results_infos.drop(["馬名"], axis=1, inplace=True)
race_results_infos_d = pd.get_dummies(race_results_infos)
race_results_infos_d["rank"] = race_results_infos_d["着順"].map(lambda x: x if x < 4 else 4)
train, test = split_data(race_results_infos_d, test_size=0.3)
X_train = train.drop(["着順", "date", "rank"], axis=1)
y_train = train["rank"]
X_test = test.drop(["着順", "date", "rank"], axis=1)
y_test = test["rank"]

In [18]:
#アンダーサンプリング

rank_1 = train["rank"].value_counts()[1]
rank_2 = train["rank"].value_counts()[2]
rank_3 = train["rank"].value_counts()[3]
rus = RandomUnderSampler(
    sampling_strategy={1: rank_1, 2: rank_2, 3: rank_3, 4: rank_1}, random_state=71
)

X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

In [19]:
#ランダムフォレストによる予測

clf = RandomForestClassifier(random_state=0)
clf.fit(X_train_rus, y_train_rus)
print(clf.score(X_train, y_train), clf.score(X_test, y_test))

0.7629952871356258 0.5927343813635253
